In [42]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

In [87]:


sta1_RA_WORLD = 174.77413750 - 9*10**(-5) #HD 101274
sta1_DEC_WORLD = -37.75375000 + 6.4*10**(-5)
sta2_RA_WORLD = 174.82283333 - 0.15/3600. #Tycho-2 7740-22-1
sta2_DEC_WORLD = -37.76239167
sta3_RA_WORLD = 174.79553333 #174.79553250 #Tycho-2 7740-529-1
sta3_DEC_WORLD = -37.83470278 #-37.83470333

k_g = 0.16
k_r = 0.08
k_i = 0.02
k_z = 0.01

PG_RA = 201.4144438
PG_DEC = -8.8219767

RA_SA100_241 = 133.14186958
DEC_SA100_241 = -0.66360306

RA_SA101_315 = 148.71371625  
DEC_SA101_315 = -0.45861306

P_mag_g = 12.225
P_mag_r = 11.918
P_mag_i = 11.829
P_mag_z = 11.814

'''
g_SA100_241 = 10.166
r_SA100_241 = 10.229
i_SA100_241 = 10.347
z_SA100_241 = 10.456

g_SA101_315 = 11.703
r_SA101_315 = 10.862
i_SA101_315 = 10.560
z_SA101_315 = 10.416
'''

g_P_RA = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
g_P_DEC = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).dec.degree 

r_P_RA = SkyCoord('13:26:05 -08:50:37', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
r_P_DEC = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).dec.degree 

i_P_RA = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
i_P_DEC = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).dec.degree 

#z_P_RA = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
#z_P_DEC = SkyCoord('13:26:05 -08:50:36', unit=(u.hourangle, u.deg)).dec.degree 

z_P_RA = SkyCoord('13:26:08 -08:49:15', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
z_P_DEC = SkyCoord('13:26:08 -08:49:15', unit=(u.hourangle, u.deg)).dec.degree 


In [45]:
NGC3783_g = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-g',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-g')]

P_g = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-g',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-g/')]

In [46]:
NGC3783_r = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-r',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-r')]

P_r = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-r',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-r/')]

In [47]:
NGC3783_i = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-i',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-i')]

P_i = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-i',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-i/')]

In [48]:
NGC3783_z = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/NGC3783-z')]

P_z = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783-cal/15-07-17/PG1323-086-z/')]

In [49]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [50]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [51]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [52]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [53]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [54]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [77]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":50, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [89]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 20. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 20. and \
            DECmain - 20. < out['table']['YMODEL_IMAGE'][j] < DECmain + 20.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [99]:
sta1_RA_WORLD = 174.77413750 - 9*10**(-5) #HD 101274
sta1_DEC_WORLD = -37.75375000 + 6.4*10**(-5)
sta2_RA_WORLD = 174.82283333 - 0.15/3600. #Tycho-2 7740-22-1
sta2_DEC_WORLD = -37.76239167
sta3_RA_WORLD = 174.79553333 #174.79553250 #Tycho-2 7740-529-1
sta3_DEC_WORLD = -37.83470278 #-37.83470333
qua1_RA_WORLD = 174.75717083
qua1_DEC_WORLD = -37.73850000



def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    aperture1 = 0.001
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        print 'Frame nr. ', i
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/header['EXPTIME']
        if a1 > 0:
            print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        print 'Frame nr. ', i, 
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/header['EXPTIME']
        if a1 > 0:
            print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu/com_EXPTIME
    star_flu = star_flu/star_EXPTIME
    com_flux = np.mean(com_flu)
    com_AIRMASS_num = np.mean(com_AIRMASS)
    star_AIRMASS_num = np.mean(star_AIRMASS)
    mag = -(AIR_EXTINCTION)*(star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/float(com_flux))
    print mag
    return mag

In [97]:
HD101274_1 = findmag(NGC3783_g,P_g,sta1_RA_WORLD,sta1_DEC_WORLD,g_P_RA,g_P_DEC,P_mag_g,k_g)
Tyc_22_1 = findmag(NGC3783_g,P_g,sta2_RA_WORLD,sta2_DEC_WORLD,g_P_RA,g_P_DEC,P_mag_g,k_g)
Tyc_529_1 = findmag(NGC3783_g,P_g,sta3_RA_WORLD,sta3_DEC_WORLD,g_P_RA,g_P_DEC,P_mag_g,k_g)




Frame nr.  0
6012.55
Frame nr.  1
3536.315
Frame nr.  2
6697.45
Frame nr.  3
5893.935
Frame nr.  4
5468.68
Frame nr.  5
4740.82
Frame nr.  6
5837.075
Frame nr.  7
6278.025
Frame nr.  0INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that

4740.82
Frame nr.  6
5837.075
Frame nr.  7
6278.025
Frame nr.  0INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS 

In [98]:
print 'g-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

g-filter
[ 9.90489003  9.91535866]
9.9101243453 0.00523431705519
[ 13.10991382  13.10594429]
13.1079290548 0.00198476973675
[]
nan nan


/home/lynge/.local/lib/python2.7/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/lynge/.local/lib/python2.7/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/lynge/.local/lib/python2.7/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [64]:
HD101274_1 = findmag(NGC3783_r,P_r,sta1_RA_WORLD,sta1_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_22_1 = findmag(NGC3783_r,P_r,sta2_RA_WORLD,sta2_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)
Tyc_529_1 = findmag(NGC3783_r,P_r,sta3_RA_WORLD,sta3_DEC_WORLD,r_P_RA,r_P_DEC,P_mag_r,k_r)




Frame nr.  0
Frame nr.  1
Frame nr.  2
Frame nr.  3
Frame nr.  4
Frame nr.  5
Frame nr.  6
Frame nr.  7
Frame nr.  0INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please appen

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

In [65]:
print 'r-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

r-filter
[ 10.14427905  10.1042977 ]
10.1242883764 0.019990675644
[ 12.72041131  12.68426239]
12.7023368481 0.0180744594805
[ nan  nan]
nan nan


In [66]:
HD101274_1 = findmag(NGC3783_i,P_i,sta1_RA_WORLD,sta1_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_22_1 = findmag(NGC3783_i,P_i,sta2_RA_WORLD,sta2_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)
Tyc_529_1 = findmag(NGC3783_i,P_i,sta3_RA_WORLD,sta3_DEC_WORLD,i_P_RA,i_P_DEC,P_mag_i,k_i)




Frame nr.  0
Frame nr.  1
Frame nr.  2
Frame nr.  3
Frame nr.  4
Frame nr.  5
Frame nr.  6
Frame nr.  7
Frame nr.  0INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please appen

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

In [67]:
print 'i-filter'
print HD101274_1
print np.mean(HD101274_1,), np.std(HD101274_1)

print Tyc_22_1
print np.mean(Tyc_22_1), np.std(Tyc_22_1)

print Tyc_529_1
print np.mean(Tyc_529_1), np.std(Tyc_529_1)

i-filter
[ 10.35022018  10.29668573  10.3036334 ]
10.3168464362 0.0237686425407
[ 12.67868227  12.6412134   12.64874617]
12.6562139494 0.0161823948162
[ nan  nan  nan]
nan nan


In [100]:
sta2_RA_WORLD = 174.82083333
sta2_DEC_WORLD = -37.76194444

HD101274_1 = findmag(NGC3783_z,P_z,sta1_RA_WORLD,sta1_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_22_1 = findmag(NGC3783_z,P_z,sta2_RA_WORLD,sta2_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)
Tyc_529_1 = findmag(NGC3783_z,P_z,sta3_RA_WORLD,sta3_DEC_WORLD,z_P_RA,z_P_DEC,P_mag_z,k_z)




Frame nr.  0
Frame nr.  1
1258.49325
Frame nr.  2
Frame nr.  3
1230.11675
Frame nr.  4
Frame nr.  5
Frame nr.  6
Frame nr.  7
Frame nr.  0INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for co

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

In [101]:
print 'z-filter'
print HD101274_1
print np.nanmean(HD101274_1,), np.nanstd(HD101274_1)

print Tyc_22_1
print np.nanmean(Tyc_22_1), np.nanstd(Tyc_22_1)

print Tyc_529_1
print np.nanmean(Tyc_529_1), np.nanstd(Tyc_529_1)

z-filter
[ 10.13643602  10.1015382 ]
10.1189871086 0.0174489126791
[]
nan nan
[]
nan nan


/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  
/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
